In [103]:
import os
import pandas as pd
import ast
from pandarallel import pandarallel

In [104]:
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [112]:
df_push_raw = pd.read_csv('drive/output/scrape/extract_github_data/push_data/push_2020_1.csv', index_col = 0)
df_push_commits = pd.read_csv('drive/output/scrape/push_pr_commit_data/push_data/push_data_2020_1.csv', index_col = 0)
df_pull_request_raw = pd.read_csv('drive/output/scrape/extract_github_data/pull_request_data/pull_request_2020_1.csv', index_col = 0)
df_pull_request_commits = pd.read_csv('drive/output/scrape/push_pr_commit_data/pull_request_data/pull_request_data_2020_1.csv', index_col = 0)

/tmp/user/20506/ipykernel_3232510/2217606071.py:3: DtypeWarning: Columns (31,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pull_request_raw = pd.read_csv('drive/output/scrape/extract_github_data/pull_request_data/pull_request_2020_1.csv', index_col = 0)


In [113]:
df_push = pd.merge(df_push_raw.drop(['commit_urls'], axis = 1), df_push_commits, how = 'left', on = ['push_id'])
df_pull_request = pd.concat([df_pull_request_raw,df_pull_request_commits[['commit_list']]], axis = 1)

In [ ]:
# problem is that some pull requests are also pushes

In [114]:
df_pull_request['commit_list'] = df_pull_request['commit_list'].apply(ast.literal_eval)
df_push['commit_urls'] = df_push['commit_urls'].apply(lambda x: ast.literal_eval(x.replace('\n', ', ')))
df_push['commit_list'] = df_push['commit_urls'].apply(lambda x: [ele.split("/")[-1] for ele in x])

In [115]:
df_push = df_push[df_push['commit_list'].apply(len) != 0]

In [116]:
df_push['potential_merge_commit'] = df_push['commit_list'].apply(lambda x: x[-1])
df_push['potential_pr_commits'] = df_push['commit_list'].apply(lambda x: set(x[:-1]))
df_pull_request['commit_list'] = df_pull_request['commit_list'].apply(lambda x: set(x))

In [117]:
df_pull_request[df_pull_request['repo_name'] == 'pypa/pip']

,type,created_at,repo_id,repo_name,actor_id,actor_login,org_id,org_login,pr_id,pr_node_id,pr_title,pr_state,pr_locked,pr_number,pr_body,pr_issue_url,pr_merged_at,pr_closed_at,pr_updated_at,pr_commits,pr_additions,pr_deletions,pr_changed_files,pr_author_association,pr_assignee,pr_assignees,pr_requested_reviewers,pr_requested_teams,pr_ref,pr_action,pr_merged_by_login,pr_merged_by_id,pr_merged_by_type,pr_merged_by_site_admin,pr_label,pr_patch_url,pr_commits_url,commit_list
1,PullRequestEvent,2020-01-29 22:43:21+00:00,1446467,pypa/pip,2312275,chrahunt,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,closed,chrahunt,2312275.0,User,False,['trivial'],https://github.com/pypa/pip/pull/7672.patch,https://api.github.com/repos/pypa/pip/pulls/76...,{dd8753cdeef9d909ba255679eeccb20c5e5353d7}
2,PullRequestEvent,2020-01-04 22:06:14+00:00,1446467,pypa/pip,1089744,xavfernandez,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,closed,xavfernandez,1089744.0,User,False,['trivial' 'type: refactor'],https://github.com/pypa/pip/pull/7553.patch,https://api.github.com/repos/pypa/pip/pulls/75...,"{f9bf1a70046afc72de66490cac1a1d4b72eca621, b65..."
3,PullRequestEvent,2020-01-20 09:41:50+00:00,1446467,pypa/pip,1089744,xavfernandez,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,closed,xavfernandez,1089744.0,User,False,['trivial' 'type: refactor'],https://github.com/pypa/pip/pull/7584.patch,https://api.github.com/repos/pypa/pip/pulls/75...,{8848e4a0a3604a78e7283cf386611c7ee2e351c0}
534,PullRequestEvent,2020-01-11 23:48:21+00:00,1446467,pypa/pip,2312275,chrahunt,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,closed,chrahunt,2312275.0,User,False,['trivial'],https://github.com/pypa/pip/pull/7580.patch,https://api.github.com/repos/pypa/pip/pulls/75...,{99f582185e3ef5a7264d319fa960aadb55df982b}
1022,PullRequestEvent,2020-01-31 09:39:03+00:00,1446467,pypa/pip,3275593,pradyunsg,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,closed,pradyunsg,3275593.0,User,False,['trivial'],https://github.com/pypa/pip/pull/7662.patch,https://api.github.com/repos/pypa/pip/pulls/76...,{96ce5236f1a9b204d4ef38cf859c1779b4b338f0}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40608,PullRequestEvent,2020-01-29 03:42:02+00:00,1446467,pypa/pip,2312275,chrahunt,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opened,NaN,NaN,NaN,NaN,[],https://github.com/pypa/pip/pull/7668.patch,https://api.github.com/repos/pypa/pip/pulls/76...,{8e59ad33b240b5ded2f3d510a1847593a7bb0bb1}
40609,PullRequestEvent,2020-01-02 22:22:16+00:00,1446467,pypa/pip,2312275,chrahunt,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opened,NaN,NaN,NaN,NaN,['C: tests' 'trivial' 'type: refactor'],https://github.com/pypa/pip/pull/7547.patch,https://api.github.com/repos/pypa/pip/pulls/75...,{2ff8de291b0efe8bfbc6c1c694a64ca78efabad1}
40610,PullRequestEvent,2020-01-06 23:29:30+00:00,1446467,pypa/pip,2312275,chrahunt,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opened,NaN,NaN,NaN,NaN,['trivial' 'type: refactor'],https://github.com/pypa/pip/pull/7562.patch,https://api.github.com/repos/pypa/pip/pulls/75...,"{893faa9e449eecdaf09b91bc736bacf0dc2af7df, 4b5..."
40611,PullRequestEvent,2020-01-07 10:57:43+00:00,1446467,pypa/pip,605277,uranusjr,647025.0,pypa,NaN,NaN,NaN,NaN,NaN,7532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reopened,NaN,NaN,NaN,NaN,[],https://github.com/pypa/pip/pull/7532.patch,https://api.github.com/repos/pypa/pip/pulls/75...,"{6c2ffcd0ce7f1d48a14017e61c9cd9cd0a0ba3dc, ed4..."
